In [80]:
#Imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import numpy as np


**Process Data on Environmental Taxes**

In [81]:
#Set working directory
os.chdir('/Users/khangphamgia/Downloads')
#Load data
df_tax = pd.read_csv("/Users/khangphamgia/Downloads/Environmental_Taxes.csv")
#Drop multiple unncessary columns
df_tax = df_tax.drop(columns = ["ObjectId", "ISO2", "ISO3", "Source", "CTS Code", "CTS Name", "CTS Full Descriptor", "1995", "1996", "1997", "1998", "1999","2000", "2020", "2021"])
df_tax.head()

,Country,Indicator,Unit,2001,2002,2003,2004,2005,2006,2007,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Albania,Environmental Taxes,Domestic Currency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.399314e+10,4.781379e+10,4.754858e+10,5.114559e+10,5.341565e+10
1,Albania,Environmental Taxes,Percent of GDP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.067206e+00,3.247163e+00,3.066373e+00,3.124865e+00,3.157133e+00
2,Albania,Taxes on Energy (including fuel for transport),Domestic Currency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.774111e+10,4.094562e+10,4.040004e+10,4.352182e+10,4.516530e+10
3,Albania,Taxes on Energy (including fuel for transport),Percent of GDP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.631314e+00,2.780726e+00,2.605369e+00,2.659072e+00,2.669496e+00
4,Albania,Taxes on Pollution,Domestic Currency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.782069e+09,1.879970e+09,1.941324e+09,2.226251e+09,2.625011e+09


In [82]:
# Select Environmental Taxes
df_tax = df_tax[df_tax['Indicator'] == 'Environmental Taxes']
# Search for unique countries
df_tax["Country"].unique()

array(['Albania', 'Andorra, Principality of', 'Antigua and Barbuda',
       'Argentina', 'Armenia, Rep. of', 'Australia', 'Austria',
       'Bahamas, The', 'Bangladesh', 'Belgium', 'Belize', 'Bhutan',
       'Bolivia', 'Botswana', 'Brazil', 'Bulgaria', 'Burkina Faso',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Chad', 'Chile',
       'China, P.R.: Mainland', 'Colombia', 'Congo, Dem. Rep. of the',
       'Congo, Rep. of', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire",
       'Croatia, Rep. of', 'Cyprus', 'Czech Rep.', 'Denmark',
       'Dominican Rep.', 'Ecuador', 'Egypt, Arab Rep. of', 'El Salvador',
       'Equatorial Guinea, Rep. of', 'Estonia, Rep. of',
       'Eswatini, Kingdom of', 'Fiji, Rep. of', 'Finland', 'France',
       'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guyana',
       'Honduras', 'Hungary', 'Iceland', 'Ireland', 'Israel', 'Italy',
       'Jamaica', 'Japan', 'Kazakhstan, Rep. of', 'Kenya',
       'Korea, Rep. of', 'Kosovo, Rep. of', 'Kyrgyz R

In [83]:
print(df_tax)

                             Country            Indicator               Unit  \
0                            Albania  Environmental Taxes  Domestic Currency   
1                            Albania  Environmental Taxes     Percent of GDP   
10          Andorra, Principality of  Environmental Taxes  Domestic Currency   
15               Antigua and Barbuda  Environmental Taxes  Domestic Currency   
16               Antigua and Barbuda  Environmental Taxes     Percent of GDP   
...                              ...                  ...                ...   
1210  Venezuela, Rep. Bolivariana de  Environmental Taxes     Percent of GDP   
1219                         Vietnam  Environmental Taxes  Domestic Currency   
1220                         Vietnam  Environmental Taxes     Percent of GDP   
1229              West Bank and Gaza  Environmental Taxes  Domestic Currency   
1230              West Bank and Gaza  Environmental Taxes     Percent of GDP   

             2001         2002         

In [84]:
# Step 1: Melt the dataframe to reshape year columns into rows
df_melted = df_tax.melt(id_vars=['Country', 'Indicator', 'Unit'], var_name='Year', value_name='Value')

# Step 2: Convert the Year column to numeric
df_melted['Year'] = pd.to_numeric(df_melted['Year'], errors='coerce')

# Step 3: Pivot the dataframe to get the required format
df_pivot = df_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Unit'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Step 4: Rename columns for clarity
df_pivot.columns.name = None  # Remove multi-index column name
df_pivot.rename(columns={
    'Percent of GDP': 'Environmental Taxes (% of GDP)',
    'Domestic Currency': 'Environmental Taxes (Domestic Currency)'
}, inplace=True)

# Step 5: Create a full range of countries and years
all_countries = df_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2021))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
df_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
df_main = pd.merge(df_full, df_pivot, on=['Country', 'Year'], how='left')

# Step 6: Fill missing values with NaN or placeholders if necessary
df_main['Environmental Taxes (% of GDP)'] = df_main['Environmental Taxes (% of GDP)'].fillna(np.nan)
df_main['Environmental Taxes (Domestic Currency)'] = df_main['Environmental Taxes (Domestic Currency)'].fillna(np.nan)

# Final dataframe
print(df_main)

#Export to excel 
#df_main.to_excel("Environmental_Taxes_Cleaned.xlsx", index=False)



                 Country  Year  Environmental Taxes (Domestic Currency)  \
0                Albania  2000                                      NaN   
1                Albania  2001                                      NaN   
2                Albania  2002                                      NaN   
3                Albania  2003                                      NaN   
4                Albania  2004                                      NaN   
...                  ...   ...                                      ...   
2641  West Bank and Gaza  2016                             2.752039e+07   
2642  West Bank and Gaza  2017                             2.752039e+07   
2643  West Bank and Gaza  2018                             2.752039e+07   
2644  West Bank and Gaza  2019                             2.752039e+07   
2645  West Bank and Gaza  2020                                      NaN   

      Environmental Taxes (% of GDP)  
0                                NaN  
1                    

In [85]:
# Run summary statistics for each country
summary_stats = df_main.groupby('Country').agg({
    'Environmental Taxes (% of GDP)': ['mean', 'median', 'std', 'min', 'max', 'count'],
    'Environmental Taxes (Domestic Currency)': ['mean', 'median', 'std', 'min', 'max', 'count']
}).reset_index()
print(summary_stats)

                            Country Environmental Taxes (% of GDP)            \
                                                              mean    median   
0                           Albania                       3.132548  3.124865   
1          Andorra, Principality of                            NaN       NaN   
2               Antigua and Barbuda                       0.537895  0.630000   
3                         Argentina                       1.234211  1.220000   
4                  Armenia, Rep. of                       1.034888  1.049627   
..                              ...                            ...       ...   
121                   United States                       0.791067  0.752718   
122                         Uruguay                       1.674216  1.670845   
123  Venezuela, Rep. Bolivariana de                       2.177000  1.640000   
124                         Vietnam                       0.469412  0.430000   
125              West Bank and Gaza     

In [86]:
#Create a column for value count of each country for 2 variables in the original dataframe
df_main['count'] = df_main.groupby('Country')['Environmental Taxes (% of GDP)'].transform('count')
#Drop countries with less than 10 observations
df_main = df_main[df_main['count'] >= 10]
#Drop the count column
df_main = df_main.drop(columns = ['count'])
#Print the new dataframe
df_main["Country"].unique()


array(['Antigua and Barbuda', 'Argentina', 'Australia', 'Austria',
       'Bahamas, The', 'Bangladesh', 'Belgium', 'Belize', 'Bolivia',
       'Bulgaria', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Chad',
       'China, P.R.: Mainland', 'Congo, Dem. Rep. of the',
       'Congo, Rep. of', 'Costa Rica', 'Croatia, Rep. of', 'Cyprus',
       'Czech Rep.', "Côte d'Ivoire", 'Denmark', 'Ecuador',
       'Egypt, Arab Rep. of', 'Estonia, Rep. of', 'Eswatini, Kingdom of',
       'Fiji, Rep. of', 'Finland', 'France', 'Germany', 'Ghana', 'Greece',
       'Guyana', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Jamaica',
       'Japan', 'Kenya', 'Korea, Rep. of', 'Kyrgyz Rep.', 'Latvia',
       'Lithuania', 'Luxembourg', 'Malaysia', 'Maldives', 'Mali', 'Malta',
       'Mauritania, Islamic Rep. of', 'Mauritius', 'Mexico', 'Mongolia',
       'Morocco', 'Namibia', 'Netherlands, The', 'Nicaragua', 'Niger',
       'Nigeria', 'Norway', 'Panama', 'Papua New Guinea', 'Paraguay',
       'Peru', 'Philippines', 'Pola

In [87]:
import pandas as pd

# List of countries
countries = [
    'Antigua and Barbuda', 'Argentina', 'Australia', 'Austria',
    'Bahamas, The', 'Bangladesh', 'Belgium', 'Belize', 'Bolivia',
    'Bulgaria', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Chad',
    'China, P.R.: Mainland', 'Congo, Dem. Rep. of the',
    'Congo, Rep. of', 'Costa Rica', 'Croatia, Rep. of', 'Cyprus',
    'Czech Rep.', "Côte d'Ivoire", 'Denmark', 'Ecuador',
    'Egypt, Arab Rep. of', 'Estonia, Rep. of', 'Eswatini, Kingdom of',
    'Fiji, Rep. of', 'Finland', 'France', 'Germany', 'Ghana', 'Greece',
    'Guyana', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Jamaica',
    'Japan', 'Kenya', 'Korea, Rep. of', 'Kyrgyz Rep.', 'Latvia',
    'Lithuania', 'Luxembourg', 'Malaysia', 'Maldives', 'Mali', 'Malta',
    'Mauritania, Islamic Rep. of', 'Mauritius', 'Mexico', 'Mongolia',
    'Morocco', 'Namibia', 'Netherlands, The', 'Nicaragua', 'Niger',
    'Nigeria', 'Norway', 'Panama', 'Papua New Guinea', 'Paraguay',
    'Peru', 'Philippines', 'Poland, Rep. of', 'Portugal', 'Romania',
    'Rwanda', 'Serbia, Rep. of', 'Seychelles', 'Sierra Leone',
    'Slovak Rep.', 'Slovenia, Rep. of', 'Solomon Islands',
    'South Africa', 'St. Lucia', 'Sweden', 'Switzerland', 'Togo',
    'Trinidad and Tobago', 'Tunisia', 'Türkiye, Rep. of', 'Uganda',
    'United Kingdom', 'United States',
    'Venezuela, Rep. Bolivariana de', 'Vietnam'
]

# Country-region mapping
region_mapping = {
    'North America': ['Antigua and Barbuda', 'Bahamas, The', 'Belize', 'Canada', 'Costa Rica', 'Jamaica', 
                      'Mexico', 'Panama', 'St. Lucia', 'Trinidad and Tobago', 'United States'],
    'South America': ['Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 
                      'Peru', 'Suriname', 'Uruguay', 'Venezuela, Rep. Bolivariana de'],
    'Europe': ['Austria', 'Belgium', 'Bulgaria', 'Croatia, Rep. of', 'Cyprus', 'Czech Rep.', 'Denmark', 
               'Estonia, Rep. of', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 
               'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands, The', 'Norway', 'Poland, Rep. of', 
               'Portugal', 'Romania', 'Serbia, Rep. of', 'Slovak Rep.', 'Slovenia, Rep. of', 'Spain', 'Sweden', 
               'Switzerland', 'United Kingdom'],
    'Asia': ['Bangladesh', 'Cambodia', 'China, P.R.: Mainland', 'India', 'Japan', 'Korea, Rep. of', 'Kyrgyz Rep.', 
             'Malaysia', 'Maldives', 'Mongolia', 'Philippines', 'Singapore', 'Thailand', 'Vietnam'],
    'Africa': ['Cameroon', 'Cabo Verde', 'Chad', 'Congo, Dem. Rep. of the', 'Congo, Rep. of', 
               'Egypt, Arab Rep. of', 'Eswatini, Kingdom of', 'Ghana', 'Kenya', 'Mali', 'Mauritania, Islamic Rep. of', 
               'Mauritius', 'Morocco', 'Namibia', 'Nigeria', 'Rwanda', 'Seychelles', 'Sierra Leone', 'South Africa', 
               'Togo', 'Tunisia', 'Uganda'],
    'Oceania': ['Australia', 'Fiji, Rep. of', 'Papua New Guinea', 'Solomon Islands'],
}

df_main['Region'] = df_main['Country'].map({country: region for region, countries in region_mapping.items() for country in countries})
df_main.head()





,Country,Year,Environmental Taxes (Domestic Currency),Environmental Taxes (% of GDP),Region
42,Antigua and Barbuda,2000,NaN,NaN,North America
43,Antigua and Barbuda,2001,19240000.0,0.89,North America
44,Antigua and Barbuda,2002,19340000.0,0.88,North America
45,Antigua and Barbuda,2003,18840000.0,0.82,North America
46,Antigua and Barbuda,2004,20040000.0,0.81,North America


**Process Data on Renewable Energy Share in Total Final Energy Consumption**

In [88]:
# Load the data 
renewable_energy = pd.read_excel("/Users/khangphamgia/Downloads/Renewable Energy Share in Total Final Energy Consumption (%).xlsx") 
# Rename the column 'Economy' to 'Country' to match the existing dataframe
renewable_energy.rename(columns={'Economy': 'Country'}, inplace=True)
renewable_energy.head()


,Indicator,Country,Unit of Measure,2000,2001,2002,2003,2004,2005,2006,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Renewable Energy Share in Total Final Energy C...,Central and West Asia,NaN,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,Renewable Energy Share in Total Final Energy C...,Afghanistan,percent,44.99,45.6,37.83,36.66,44.24,33.88,31.89,...,19.06,17.69,20.18,19.48,18.28,18.87,18.21,19.98,...,...
2,Renewable Energy Share in Total Final Energy C...,Armenia,percent,7.16,5.42,8.9,9.81,8.71,6.49,7.65,...,7.08,10.74,13.15,12.56,11.13,10.34,8.38,9.13,...,...
3,Renewable Energy Share in Total Final Energy C...,Azerbaijan,percent,2.07,1.75,2.35,2.94,3.07,3.36,2.86,...,2.12,2.33,1.93,1.9,1.92,1.69,1.29,1.32,...,...
4,Renewable Energy Share in Total Final Energy C...,Georgia,percent,47.25,53.15,56.76,55.33,54.53,41.48,37.22,...,30.69,27.54,27.18,27.52,27.37,24.69,23.45,25.18,...,...


In [89]:
# Transform the data
#Melt the dataframe to reshape year columns into rows
renewable_energy_melted = renewable_energy.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
renewable_energy_melted['Year'] = pd.to_numeric(renewable_energy_melted['Year'], errors='coerce')

# Step 3: Pivot the dataframe to get the required format
renewable_energy_pivot = renewable_energy_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Unit of Measure'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
renewable_energy_pivot.columns.name = None  # Remove multi-index column name
renewable_energy_pivot.rename(columns={
    'percent': 'Renewable Energy Share in Total Final Energy Consumption (%)'
}, inplace=True)

# Create a full range of countries and years
all_countries = renewable_energy_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2022))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
renewable_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
renewable_main = pd.merge(renewable_full, renewable_energy_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
renewable_main['Renewable Energy Share in Total Final Energy Consumption (%)'] = renewable_main['Renewable Energy Share in Total Final Energy Consumption (%)'].fillna(np.nan)

print(renewable_main)

          Country  Year  \
0     Afghanistan  2000   
1     Afghanistan  2001   
2     Afghanistan  2002   
3     Afghanistan  2003   
4     Afghanistan  2004   
...           ...   ...   
1073     Viet Nam  2017   
1074     Viet Nam  2018   
1075     Viet Nam  2019   
1076     Viet Nam  2020   
1077     Viet Nam  2021   

     Renewable Energy Share in Total Final Energy Consumption (%)  
0                                                 44.99            
1                                                  45.6            
2                                                 37.83            
3                                                 36.66            
4                                                 44.24            
...                                                 ...            
1073                                              28.32            
1074                                              24.34            
1075                                              20.43        

**Process Data on Household' Final Consumption Expenditure**

In [90]:
# Load the data
household_cons = pd.read_excel("/Users/khangphamgia/Downloads/Household Consumption Expenditure (% of GDP).xlsx")
# Rename the column 'Economy' to 'Country' to match the existing dataframe
household_cons.rename(columns={'Economy': 'Country'}, inplace=True)

In [91]:
# Transform the data
household_cons_melted = household_cons.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
household_cons_melted['Year'] = pd.to_numeric(household_cons_melted['Year'], errors='coerce')

# Step 3: Pivot the dataframe to get the required format
household_cons_pivot = household_cons_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Unit of Measure'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
household_cons_pivot.columns.name = None  # Remove multi-index column name
household_cons_pivot.rename(columns={
    'percent of GDP': 'Household Consumption Expenditure (% of GDP)'
}, inplace=True)

# Create a full range of countries and years
all_countries = household_cons_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
household_cons_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
household_cons_main = pd.merge(household_cons_full, household_cons_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
household_cons_main['Household Consumption Expenditure (% of GDP)'] = household_cons_main['Household Consumption Expenditure (% of GDP)'].fillna(np.nan)

print(household_cons_main)

          Country  Year Household Consumption Expenditure (% of GDP)
0     Afghanistan  2000                                          ...
1     Afghanistan  2001                                          ...
2     Afghanistan  2002                                   111.175303
3     Afghanistan  2003                                   126.551644
4     Afghanistan  2004                                   120.511733
...           ...   ...                                          ...
1171     Viet Nam  2019                                     56.82807
1172     Viet Nam  2020                                    55.967075
1173     Viet Nam  2021                                        55.52
1174     Viet Nam  2022                                        54.79
1175     Viet Nam  2023                                    54.584566

[1176 rows x 3 columns]


**Process Data on Electricity Production**

In [92]:
elec_combust = pd.read_excel("/Users/khangphamgia/Downloads/Sources of Electricity, Combustible Fuels (% of total).xlsx")
elec_hydro = pd.read_excel("/Users/khangphamgia/Downloads/Sources of Electricity, Hydropower (% of total).xlsx")
elec_solar = pd.read_excel("/Users/khangphamgia/Downloads/Sources of Electricity, Solar (% of total).xlsx")
elec_other = pd.read_excel("/Users/khangphamgia/Downloads/Sources of Electricity, Others (% of total).xlsx")
total_elec_production = pd.read_excel("/Users/khangphamgia/Downloads/Total Electricity Production (kWh billion).xlsx")

In [93]:
elec_combust.head()

,Indicator,Economy,Unit of Measure,2000,2001,2002,2003,2004,2005,2006,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"Sources of Electricity, Combustible Fuels (% o...",Central and West Asia,percent,69.688789,71.153958,69.200383,67.471898,68.260557,67.576221,68.847185,...,72.65956,71.974049,71.898018,72.822673,72.462003,70.721918,72.116388,71.533905,...,...
1,"Sources of Electricity, Combustible Fuels (% o...",Afghanistan,percent,25.010906,27.18854,20.936029,29.750271,29.109159,25.93819,29.24425,...,11.725664,11.145141,11.098357,12.412326,15.370718,11.239003,9.06571,9.226402,9.193777,...
2,"Sources of Electricity, Combustible Fuels (% o...",Armenia,percent,45.182947,48.563969,28.610255,27.649518,26.749585,28.937787,24.844302,...,42.43871,35.914861,35.283664,36.981457,43.3258,39.675347,40.383684,42.946161,...,...
3,"Sources of Electricity, Combustible Fuels (% o...",Azerbaijan,percent,91.796353,93.141441,89.198439,88.400827,87.329255,86.844176,89.740456,...,94.722582,93.32999,91.915168,92.575521,92.50868,93.42458,95.307101,94.890777,...,...
4,"Sources of Electricity, Combustible Fuels (% o...",Georgia,percent,21.06681,20.109557,6.752101,8.854749,12.637204,14.187423,29.069614,...,19.631665,21.95892,19.315344,19.364854,17.40859,23.955873,25.276439,18.818357,23.779184,...


In [94]:
elec_hydro.rename(columns={'Economy': 'Country'}, inplace=True)
# Transform the data
elec_hydro_melted = elec_hydro.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
elec_hydro_melted['Year'] = pd.to_numeric(elec_hydro_melted['Year'], errors='coerce')

# Step 3: Pivot the dataframe to get the required format
elec_hydro_pivot = elec_hydro_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Unit of Measure'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
elec_hydro_pivot.columns.name = None  # Remove multi-index column name
elec_hydro_pivot.rename(columns={
    'percent': 'Sources of Electricity, Hydropower (% of Total)'
}, inplace=True)

# Create a full range of countries and years
all_countries = elec_hydro_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
elec_hydro_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
elec_hydro_main = pd.merge(elec_hydro_full,elec_hydro_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
elec_hydro_main['Sources of Electricity, Hydropower (% of Total)'] = elec_hydro_main['Sources of Electricity, Hydropower (% of Total)'].fillna(np.nan)

print(elec_hydro_main)


          Country  Year Sources of Electricity, Hydropower (% of Total)
0     Afghanistan  2000                                       74.989094
1     Afghanistan  2001                                        72.81146
2     Afghanistan  2002                                       79.063971
3     Afghanistan  2003                                       70.249729
4     Afghanistan  2004                                       70.890841
...           ...   ...                                             ...
1003     Viet Nam  2019                                        36.58481
1004     Viet Nam  2020                                       30.539689
1005     Viet Nam  2021                                       31.019131
1006     Viet Nam  2022                                             ...
1007     Viet Nam  2023                                             ...

[1008 rows x 3 columns]


In [95]:
elec_combust.rename(columns={'Economy': 'Country'}, inplace=True)
# Transform the data
elec_combust_melted = elec_combust.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
elec_combust_melted['Year'] = pd.to_numeric(elec_combust_melted['Year'], errors='coerce')

# Step 3: Pivot the dataframe to get the required format
elec_combust_pivot = elec_combust_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Unit of Measure'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
elec_combust_pivot.columns.name = None  # Remove multi-index column name
elec_combust_pivot.rename(columns={
    'percent': 'Sources of Electricity, Combustible Fuels (% of Total)'
}, inplace=True)

# Create a full range of countries and years
all_countries = elec_combust_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
elec_combust_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
elec_combust_main = pd.merge(elec_combust_full,elec_combust_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
elec_combust_main['Sources of Electricity, Combustible Fuels (% of Total)'] = elec_combust_main['Sources of Electricity, Combustible Fuels (% of Total)'].fillna(np.nan)

print(elec_combust_main)

          Country  Year Sources of Electricity, Combustible Fuels (% of Total)
0     Afghanistan  2000                                          25.010906    
1     Afghanistan  2001                                           27.18854    
2     Afghanistan  2002                                          20.936029    
3     Afghanistan  2003                                          29.750271    
4     Afghanistan  2004                                          29.109159    
...           ...   ...                                                ...    
1291     Viet Nam  2019                                           61.13447    
1292     Viet Nam  2020                                          65.132597    
1293     Viet Nam  2021                                          56.697952    
1294     Viet Nam  2022                                                ...    
1295     Viet Nam  2023                                                ...    

[1296 rows x 3 columns]


In [96]:
elec_solar.rename(columns={'Economy': 'Country'}, inplace=True)
# Transform the data
elec_solar_melted = elec_solar.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
elec_solar_melted['Year'] = pd.to_numeric(elec_solar_melted['Year'], errors='coerce')

# Step 3: Pivot the dataframe to get the required format
elec_solar_pivot = elec_solar_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Unit of Measure'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
elec_solar_pivot.columns.name = None  # Remove multi-index column name
elec_solar_pivot.rename(columns={
    'percent': 'Sources of Electricity, Solar (% of Total)'
}, inplace=True)

# Create a full range of countries and years
all_countries = elec_solar_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
elec_solar_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
elec_solar_main = pd.merge(elec_solar_full,elec_solar_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
elec_solar_main['Sources of Electricity, Solar (% of Total)'] = elec_solar_main['Sources of Electricity, Solar (% of Total)'].fillna(np.nan)

print(elec_solar_main)

          Country  Year Sources of Electricity, Solar (% of Total)
0     Afghanistan  2000                                        ...
1     Afghanistan  2001                                        ...
2     Afghanistan  2002                                        ...
3     Afghanistan  2003                                        ...
4     Afghanistan  2004                                        ...
...           ...   ...                                        ...
1171     Viet Nam  2019                                   2.146585
1172     Viet Nam  2020                                   3.921964
1173     Viet Nam  2021                                  10.962988
1174     Viet Nam  2022                                        ...
1175     Viet Nam  2023                                        ...

[1176 rows x 3 columns]


In [97]:
elec_other.rename(columns={'Economy': 'Country'}, inplace=True)
# Transform the data
elec_other_melted = elec_other.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
elec_other_melted['Year'] = pd.to_numeric(elec_other_melted['Year'], errors='coerce')

# Step 3: Pivot the dataframe to get the required format
elec_other_pivot = elec_other_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Unit of Measure'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
elec_other_pivot.columns.name = None  # Remove multi-index column name
elec_other_pivot.rename(columns={
    'percent': 'Sources of Electricity, Others (% of Total)'
}, inplace=True)

# Create a full range of countries and years
all_countries = elec_other_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
elec_other_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
elec_other_main = pd.merge(elec_other_full,elec_other_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
elec_other_main['Sources of Electricity, Others (% of Total)'] = elec_other_main['Sources of Electricity, Others (% of Total)'].fillna(np.nan)

print(elec_other_main)

      Country  Year Sources of Electricity, Others (% of Total)
0     Armenia  2000                                   33.652232
1     Armenia  2001                                   34.586597
2     Armenia  2002                                    41.34807
3     Armenia  2003                                   36.320669
4     Armenia  2004                                   39.850746
..        ...   ...                                         ...
811  Viet Nam  2019                                    0.134135
812  Viet Nam  2020                                    0.405749
813  Viet Nam  2021                                    1.319929
814  Viet Nam  2022                                         ...
815  Viet Nam  2023                                         ...

[816 rows x 3 columns]


In [98]:
total_elec_production.head()

,Indicator,Economy,Unit of Measure,2000,2001,2002,2003,2004,2005,2006,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Total Electricity Production (kWh billion),Central and West Asia,kilowatt-hour,237102700000,243261100000,254610311100,271844944400,282786944400,294432055600,307448944400,...,354875880000,356576625000,375123137000,399731163000,410947782000,410262199000,425336681000,438721764000,...,...
1,Total Electricity Production (kWh billion),Afghanistan,kilowatt-hour,687700000,691100000,722200000,921000000,797000000,906000000,913000000,...,1356000000,1338700000,1393900000,1420040000,1300720000,1625500000,1370880000,1409000000,1414000000,...
2,Total Electricity Production (kWh billion),Armenia,kilowatt-hour,5958000000,5745000000,5519000000,5501000000,6030000000,6317000000,5941000000,...,7750000000,7799000000,7315000000,7765243000,7791196000,7679643000,7838683000,7879014000,...,...
3,Total Electricity Production (kWh billion),Azerbaijan,kilowatt-hour,18699000000,18969000000,18701000000,21286000000,21743000000,22872000000,24543000000,...,24728000000,24688420000,24952900000,24320900000,25229200000,26072903000,25839103000,27888801000,...,...
4,Total Electricity Production (kWh billion),Georgia,kilowatt-hour,7424000000,6937000000,7257000000,7160000000,6924000000,7267000000,7599000000,...,10371000000,10832500000,11573700000,11531200000,12148600000,11856800000,11159800000,12645100000,14246620000,...


In [99]:
total_elec_production.rename(columns={'Economy': 'Country'}, inplace=True)
# Transform the data
total_elec_production_melted = total_elec_production.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
total_elec_production_melted['Year'] = pd.to_numeric(total_elec_production_melted['Year'], errors='coerce')

# Step 3: Pivot the dataframe to get the required format
total_elec_production_pivot = total_elec_production_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Indicator'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
total_elec_production_pivot.columns.name = None  # Remove multi-index column name
total_elec_production_pivot.rename(columns={
    'Total Electricity Production (kWh billion)': 'Total Electricity Production (kWh billion)'
}, inplace=True)

# Create a full range of countries and years
all_countries = total_elec_production_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
total_elec_production_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
total_elec_production_main = pd.merge(total_elec_production_full,total_elec_production_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
total_elec_production_main['Total Electricity Production (kWh billion)'] = total_elec_production_main['Total Electricity Production (kWh billion)'].fillna(np.nan)

print(total_elec_production_main)

          Country  Year Total Electricity Production (kWh billion)
0     Afghanistan  2000                                  687700000
1     Afghanistan  2001                                  691100000
2     Afghanistan  2002                                  722200000
3     Afghanistan  2003                                  921000000
4     Afghanistan  2004                                  797000000
...           ...   ...                                        ...
1315     Viet Nam  2019                               232602000000
1316     Viet Nam  2020                               240542724000
1317     Viet Nam  2021                               253498401000
1318     Viet Nam  2022                                        ...
1319     Viet Nam  2023                                        ...

[1320 rows x 3 columns]


In [100]:
#Merge all dataframes
df_electricity = pd.merge(elec_combust_main, elec_hydro_main, on=['Country', 'Year'], how='left')
df_electricity = pd.merge(df_electricity, elec_solar_main, on=['Country', 'Year'], how='left')
df_electricity = pd.merge(df_electricity, elec_other_main, on=['Country', 'Year'], how='left')
df_electricity = pd.merge(df_electricity, total_elec_production_main, on=['Country', 'Year'], how='left')
# Print country Vietnam
df_electricity[df_electricity['Country'] == 'Viet Nam']

,Country,Year,"Sources of Electricity, Combustible Fuels (% of Total)","Sources of Electricity, Hydropower (% of Total)","Sources of Electricity, Solar (% of Total)","Sources of Electricity, Others (% of Total)",Total Electricity Production (kWh billion)
1272,Viet Nam,2000,45.216671,54.783329,...,...,26561000000
1273,Viet Nam,2001,40.50575,59.49425,...,...,30608000000
1274,Viet Nam,2002,49.161918,50.838082,...,...,35796000000
1275,Viet Nam,2003,53.607819,46.392181,...,...,40925000000
1276,Viet Nam,2004,61.440412,38.559588,...,...,46209000000
1277,Viet Nam,2005,68.419189,31.580811,...,...,53656000000
1278,Viet Nam,2006,66.263865,33.736135,...,...,60493000000
1279,Viet Nam,2007,65.623508,34.376492,...,...,67008000000
1280,Viet Nam,2008,64.593438,35.405199,...,0.001362,73396000000
1281,Viet Nam,2009,63.94229,36.045687,...,0.012023,83175000000


**Process Economy Table Data**

In [101]:
# Load dataset 
emission = pd.read_excel("/Users/khangphamgia/Downloads/Emission.xlsx")
employment_share = pd.read_excel("/Users/khangphamgia/Downloads/Employment_share.xlsx")
energy_intensity = pd.read_excel("/Users/khangphamgia/Downloads/Energy_Intensity.xlsx")
gdp = pd.read_excel("/Users/khangphamgia/Downloads/GDP_currentprice.xlsx")
rd = pd.read_excel("/Users/khangphamgia/Downloads/R&D_expenditure.xlsx")
gdp_structure = pd.read_excel("/Users/khangphamgia/Downloads/Structure _of_GDP.xlsx")


In [102]:
emission.head()

,Indicator,Economy,Unit of Measure,2000,2001,2002,2003,2004,2005,2006,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,"Carbon Dioxide Emissions, Per Unit of Manufact...",Afghanistan,NaN,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,"Carbon Dioxide Emissions, Per Unit of Manufact...",Armenia,kg of CO₂ equivalent per constant 2015 USD,1.892,2.048,1.749,1.864,1.66,1.885,2.005,...,0.391,0.306,0.315,0.32,0.252,0.283,0.311,...,...,...
2,"Carbon Dioxide Emissions, Per Unit of Manufact...",Australia,kg of CO₂ equivalent per constant 2015 USD,0.535,0.51,0.389,0.476,0.496,0.478,0.457,...,0.373,0.378,0.357,0.371,0.374,0.358,0.358,...,...,...
3,"Carbon Dioxide Emissions, Per Unit of Manufact...",Azerbaijan,kg of CO₂ equivalent per constant 2015 USD,2.285,1.766,2.118,1.212,0.905,0.579,0.511,...,0.77,0.937,0.568,0.363,0.556,0.555,0.445,...,...,...
4,"Carbon Dioxide Emissions, Per Unit of Manufact...",Bangladesh,kg of CO₂ equivalent per constant 2015 USD,0.309,0.422,0.433,0.465,0.326,0.345,0.351,...,0.298,0.295,0.324,0.271,0.261,0.252,0.254,...,...,...


In [103]:
emission["Indicator"].unique()

array(['Carbon Dioxide Emissions, Per Unit of Manufacturing Value-Added (kg of CO₂ equivalent per constant 2015 USD)',
       'Carbon Dioxide Emissions, Per Unit of GDP (PPP) (kg of CO₂ equivalent per constant 2017 USD)',
       "Carbon Dioxide Emissions (t '000)",
       "Total Greenhouse Gas Emissions (t '000 CO₂ equivalent)"],
      dtype=object)

In [104]:
#Rename the column 'Economy' to 'Country' to match the existing dataframe
emission.rename(columns={'Economy': 'Country'}, inplace=True)
# Transform the data
emission_melted = emission.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
emission_melted['Year'] = pd.to_numeric(emission_melted['Year'], errors='coerce')

# Pivot the dataframe to get the required format
emission_pivot = emission_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Unit of Measure'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
emission_pivot.columns.name = None
emission_pivot.rename(columns={
    'kg of CO₂ equivalent per constant 2015 USD': 'CO2 Emissions per Manufacturing Value-Added (kg per constant 2015 USD)',
    'kg of CO₂ equivalent per constant 2017 USD': 'CO2 Emissions per GDP (PPP, kg per constant 2017 USD)',
    "metric ton": "CO2 Emissions (t '000)",
    "metric tons CO₂ equivalent": "Total GHG Emissions (t '000 CO₂ equivalent)"
}, inplace=True)

# Create a full range of countries and years
all_countries = emission_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
emission_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
emission_main = pd.merge(emission_full,emission_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN
columns = ['CO2 Emissions per Manufacturing Value-Added (kg per constant 2015 USD)', 
           'CO2 Emissions per GDP (PPP, kg per constant 2017 USD)', 
           'CO2 Emissions (t \'000)', 
           'Total GHG Emissions (t \'000 CO₂ equivalent)']
for column in columns:
    emission_main[column] = emission_main[column].fillna(np.nan)
emission_main[emission_main['Country'] == 'Viet Nam']

,Country,Year,CO2 Emissions per Manufacturing Value-Added (kg per constant 2015 USD),"CO2 Emissions per GDP (PPP, kg per constant 2017 USD)",CO2 Emissions (t '000),Total GHG Emissions (t '000 CO₂ equivalent)
1104,Viet Nam,2000,1.425,0.155,51207700,135695620
1105,Viet Nam,2001,1.457,0.16,56722400,140236535.7
1106,Viet Nam,2002,1.455,0.175,66495800,152726370.8
1107,Viet Nam,2003,1.398,0.172,70903700,160509708
1108,Viet Nam,2004,1.463,0.194,85051800,179675979.9
1109,Viet Nam,2005,1.368,0.195,92368600,188782846.5
1110,Viet Nam,2006,1.093,0.187,94869700,193546133.1
1111,Viet Nam,2007,0.979,0.193,105137400,206509700.8
1112,Viet Nam,2008,1.062,0.205,117585100,217120144.3
1113,Viet Nam,2009,1.035,0.216,132290000,239357461.7


In [105]:
employment_share.head()

,Indicator,Economy,Unit of Measure,2000,2001,2002,2003,2004,2005,2006,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Employment in Services (% of total employment),Afghanistan,percent,...,24.238567,24.238315,24.238313,24.238081,...,...,...,...,...,38.593418,...,...,36.671034,...,...,...,...
1,Employment in Services (% of total employment),Armenia,percent,35.047351,35.346668,37.762111,37.504514,37.13599,37.839315,38.200293,...,48.760022,50.61618,51.948096,51.206485,55.299796,57.18358,56.041533,56.062742,...,...
2,Employment in Services (% of total employment),Australia,percent,73.673761,74.485687,74.756191,75.245707,74.97958,75.30477,75.316699,...,78.010645,77.963186,77.800812,77.517412,78.218814,77.851472,78.629591,79.084947,78.710405,...
3,Employment in Services (% of total employment),Azerbaijan,percent,48.745947,48.859022,48.925237,48.935206,48.983047,48.846712,48.844507,...,49.563319,49.33507,49.273138,49.019327,48.919676,48.9939,48.829459,48.789047,48.534241,...
4,Employment in Services (% of total employment),Bangladesh,percent,36.153846,...,...,34.311512,...,...,37.552743,...,...,36.876585,38.896382,...,...,...,...,37.666908,...,...


In [106]:
employment_share["Indicator"].unique()

array(['Employment in Services (% of total employment)',
       'Employment in Agriculture (% of total employment)',
       'Employment in Industry (% of total employment)'], dtype=object)

In [107]:
# Rename the column 'Economy' to 'Country' to match the existing dataframe
employment_share.rename(columns={'Economy': 'Country'}, inplace=True)
# Transform the data
employment_share_melted = employment_share.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
employment_share_melted['Year'] = pd.to_numeric(employment_share_melted['Year'], errors='coerce')

# Pivot the dataframe to get the required format
employment_share_pivot = employment_share_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Indicator'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
employment_share_pivot.columns.name = None
employment_share_pivot.rename(columns={
    'Services (% of total employment)': 'Employment in Services (% of total employment)',
    'Agriculture (% of total employment)': 'Employment in Agriculture (% of total employment)',
    'Industry (% of total employment)': 'Employment in Industry (% of total employment)',
}, inplace=True)

# Create a full range of countries and years
all_countries = employment_share_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
employment_share_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
employment_share_main = pd.merge(employment_share_full,employment_share_pivot, on=['Country', 'Year'], how='left')


print(employment_share_main)


          Country  Year Employment in Agriculture (% of total employment)  \
0     Afghanistan  2000                                               ...   
1     Afghanistan  2001                                         69.574145   
2     Afghanistan  2002                                         69.574208   
3     Afghanistan  2003                                         69.574667   
4     Afghanistan  2004                                         69.574784   
...           ...   ...                                               ...   
1171     Viet Nam  2019                                         34.369287   
1172     Viet Nam  2020                                         33.022388   
1173     Viet Nam  2021                                         29.064028   
1174     Viet Nam  2022                                           27.5421   
1175     Viet Nam  2023                                         26.937502   

     Employment in Industry (% of total employment)  \
0                   

In [108]:
# Load data
energy_intensity.head()

,Indicator,Economy,Unit of Measure,2000,2001,2002,2003,2004,2005,2006,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Energy Intensity Measured in Terms of Primary ...,Afghanistan,megajoules per USD constant 2017 PPP GDP,1.5,1.5,1.4,1.39,1.21,1.41,1.5,...,2.35,2.21,2.27,2.4,2.37,2.54,2.94,...,...,...
1,Energy Intensity Measured in Terms of Primary ...,Armenia,megajoules per USD constant 2017 PPP GDP,6.9,6.28,5.11,4.73,4.53,4.76,4.28,...,4.12,3.93,3.86,3.51,3.52,4.01,3.91,...,...,...
2,Energy Intensity Measured in Terms of Primary ...,Australia,megajoules per USD constant 2017 PPP GDP,6,5.71,5.54,5.57,5.45,5.33,5.4,...,4.53,4.4,4.46,4.37,4.36,4.44,4.1,...,...,...
3,Energy Intensity Measured in Terms of Primary ...,Azerbaijan,megajoules per USD constant 2017 PPP GDP,14.06,13.38,12.35,11.79,11.34,9.42,6.91,...,4.26,4.35,4.38,4.34,4.62,4.7,4.73,...,...,...
4,Energy Intensity Measured in Terms of Primary ...,Bangladesh,megajoules per USD constant 2017 PPP GDP,2.45,2.58,2.52,2.53,2.45,2.42,2.41,...,2.17,2.2,2.12,1.96,1.95,1.9,1.93,...,...,...


In [109]:
energy_intensity["Indicator"].unique()

array(['Energy Intensity Measured in Terms of Primary Energy and GDP (MJ/$ 2017 PPP GDP)'],
      dtype=object)

In [110]:
# Rename the column 'Economy' to 'Country' to match the existing dataframe
energy_intensity.rename(columns={'Economy': 'Country'}, inplace=True)
# Transform the data
energy_intensity_melted = energy_intensity.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
energy_intensity_melted['Year'] = pd.to_numeric(energy_intensity_melted['Year'], errors='coerce')

# Pivot the dataframe to get the required format
energy_intensity_pivot = energy_intensity_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Unit of Measure'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
energy_intensity_pivot.columns.name = None  # Remove multi-index column name
energy_intensity_pivot.rename(columns={
    'megajoules per USD constant 2017 PPP GDP': 'Energy Intensity Measured in Terms of Primary Energy and GDP (MJ/$ 2017 PPP GDP)'
}, inplace=True)

# Create a full range of countries and years
all_countries = energy_intensity_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
energy_intensity_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
energy_intensity_main = pd.merge(energy_intensity_full,energy_intensity_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
energy_intensity_main['Energy Intensity Measured in Terms of Primary Energy and GDP (MJ/$ 2017 PPP GDP)'] = energy_intensity_main['Energy Intensity Measured in Terms of Primary Energy and GDP (MJ/$ 2017 PPP GDP)'].fillna(np.nan)

energy_intensity_main.head()


,Country,Year,Energy Intensity Measured in Terms of Primary Energy and GDP (MJ/$ 2017 PPP GDP)
0,Afghanistan,2000,1.5
1,Afghanistan,2001,1.5
2,Afghanistan,2002,1.4
3,Afghanistan,2003,1.39
4,Afghanistan,2004,1.21


In [111]:
#Load data
gdp.head()

,Indicator,Economy,Unit of Measure,2000,2001,2002,2003,2004,2005,2006,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,GDP at current prices,Afghanistan,Afghan Afghani,...,...,202530591600,240670915300,2.673101e+11,3.277454e+11,3.751520e+11,...,1.260009e+12,1222916881000,1285460326000,1327690395000,1469596303000,1547289027605.800049,1534965529000,1251171761000,1283441411000,...
1,GDP at current prices,Armenia,Armenian Dram,1031337900000,1175877000000,1362471700000,1624642500000,1.907945e+12,2.242881e+12,2.656189e+12,...,5.043633e+12,5067293500000,5564493300000,6017035200000,6543321800000,6181902600000,6991777900000,8501449400000,9505221600000,...
2,GDP at current prices,Australia,Australian Dollar,662674000000,707402000000,756446000000,803108000000,8.640890e+11,9.254420e+11,9.995850e+11,...,1.624733e+12,1658936000000,1759378000000,1844528000000,1950042000000,1984598000000,2088901000000,2333221000000,2561228000000,...
3,GDP at current prices,Azerbaijan,Azerbaijan Manat,4718100000,5315600000,6062500000,7146500000,8.530200e+09,1.252250e+10,1.874620e+10,...,5.438000e+10,60425200000,70337800000,80092000000,81896200000,72578000000,93203200000,133972700000,123005500000,...
4,GDP at current prices,Bangladesh,Bangladeshi Taka,2370800000000,2535500000000,2732000000000,3005800000000,3.329600e+12,3.707000e+12,4.823370e+12,...,1.515802e+13,20758212000000,23243073000000,26392479000000,29514288000000,31704694000000,35301847510000,39717163680000,44908417160000,...


In [112]:
#Rename the column 'Economy' to 'Country' to match the existing dataframe
gdp.rename(columns={'Economy': 'Country'}, inplace=True)
# Transform the data
gdp_melted = gdp.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
gdp_melted['Year'] = pd.to_numeric(gdp_melted['Year'], errors='coerce')

# Pivot the dataframe to get the required format
gdp_pivot = gdp_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Indicator'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
gdp_pivot.columns.name = None  # Remove multi-index column name
gdp_pivot.rename(columns={
    'GDP at current prices': 'GDP at current prices (domestic currency)',
}, inplace=True)

# Create a full range of countries and years
all_countries = gdp_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
gdp_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
gdp_main = pd.merge(gdp_full,gdp_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
gdp_main['GDP at current prices (domestic currency)'] = gdp_main['GDP at current prices (domestic currency)'].fillna(np.nan)

gdp_main.head()


,Country,Year,GDP at current prices (domestic currency)
0,Afghanistan,2000,...
1,Afghanistan,2001,...
2,Afghanistan,2002,202530591600
3,Afghanistan,2003,240670915300
4,Afghanistan,2004,267310072200.0


In [113]:
#Load data
rd.head()

,Indicator,Economy,Unit of Measure,2000,2001,2002,2003,2004,2005,2006,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Research and Development Expenditure as a Prop...,Afghanistan,NaN,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,Research and Development Expenditure as a Prop...,Armenia,percent of GDP,0.19106,0.27987,0.25255,0.32109,0.25659,0.25761,0.24057,...,0.25002,0.23305,0.22788,0.18876,0.17854,0.2092,0.20583,0.20966,...,...
2,Research and Development Expenditure as a Prop...,Australia,percent of GDP,1.5732,...,1.74749,...,1.84902,...,2.18011,...,1.92071,...,1.87977,...,1.82892,...,...,...,...,...
3,Research and Development Expenditure as a Prop...,Azerbaijan,percent of GDP,0.336,0.34001,0.30155,0.32496,0.29831,0.21994,0.17165,...,0.22232,0.20637,0.18468,0.18416,0.20013,0.2239,0.20836,0.15147,...,...
4,Research and Development Expenditure as a Prop...,Bangladesh,NaN,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [114]:
# Rename the column 'Economy' to 'Country' to match the existing dataframe
rd.rename(columns={'Economy': 'Country'}, inplace=True)
# Transform the data
rd_melted = rd.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
rd_melted['Year'] = pd.to_numeric(rd_melted['Year'], errors='coerce')

# Pivot the dataframe to get the required format
rd_pivot = rd_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Indicator'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
rd_pivot.columns.name = None  # Remove multi-index column name
rd_pivot.rename(columns={
    'Research and Development Expenditure as a Proportion of GDP (%)': 'R&D Expenditure (% of GDP)',
}, inplace=True)

# Create a full range of countries and years
all_countries = rd_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
rd_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
rd_main = pd.merge(rd_full,rd_pivot, on=['Country', 'Year'], how='left')

# Fill missing values with NaN or placeholders if necessary
rd_main['R&D Expenditure (% of GDP)'] = rd_main['R&D Expenditure (% of GDP)'].fillna(np.nan)

rd_main.head()

,Country,Year,R&D Expenditure (% of GDP)
0,Afghanistan,2000,...
1,Afghanistan,2001,...
2,Afghanistan,2002,...
3,Afghanistan,2003,...
4,Afghanistan,2004,...


In [115]:
#Load data
gdp_structure.head()

,Indicator,Economy,Unit of Measure,2000,2001,2002,2003,2004,2005,2006,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Services (% of GDP),Afghanistan,percent of GDP,...,...,34.587794,36.644346,40.581042,38.793691,39.300022,...,54.662644,61.979425,61.527805,62.668789,58.205625,57.24092,55.049154,49.625991,47.520147,...
1,Services (% of GDP),Armenia,percent of GDP,36.53312,36.23614,35.80308,35.11808,35.067850,34.646110,35.633730,...,52.896966,54.322522,55.439319,57.620549,59.658388,58.245477,58.465202,60.982676,64.712832,...
2,Services (% of GDP),Australia,percent of GDP,69.943249,70.282955,69.886583,70.472056,70.412508,70.050630,69.196172,...,72.044831,73.370539,71.873073,71.461769,70.681305,70.749596,70.334509,67.731795,68.3455,...
3,Services (% of GDP),Azerbaijan,percent of GDP,37.5206,36.7202,34.6502,33.9533,33.438600,26.860700,23.578800,...,43.920300,42.1233,40.4543,37.7,40.1,46.2,40.626934,34.354304,42.605219,...
4,Services (% of GDP),Bangladesh,percent of GDP,49.19781,49.96327,50.85762,51.98192,52.358740,52.633650,55.594400,...,56.350000,53.49,53.75,53.37,53.15,53.42,53.3,53.067509,52.85077,...


In [116]:
gdp_structure["Indicator"].unique()

array(['Services (% of GDP)', 'Agriculture (% of GDP)',
       'Industry (% of GDP)'], dtype=object)

In [117]:
# Rename the column 'Economy' to 'Country' to match the existing dataframe
gdp_structure.rename(columns={'Economy': 'Country'}, inplace=True)
# Transform the data
gdp_structure_melted = gdp_structure.melt(id_vars=['Country', 'Indicator', 'Unit of Measure'], var_name='Year', value_name='Value')

# Convert the Year column to numeric
gdp_structure_melted['Year'] = pd.to_numeric(gdp_structure_melted['Year'], errors='coerce')

# Pivot the dataframe to get the required format
gdp_structure_pivot = gdp_structure_melted.pivot_table(
    index=['Country', 'Year'],
    columns=['Indicator'],
    values='Value',
    aggfunc='first'  # Avoid duplicates
).reset_index()

# Rename columns for clarity
gdp_structure_pivot.columns.name = None
gdp_structure_pivot.rename(columns={
    'Services (% of GDP)': 'Services (% of GDP)',
    'Agriculture (% of GDP)': 'Agriculture (% of GDP)',
    'Industry (% of GDP)': 'Industry (% of GDP)',
}, inplace=True)

# Create a full range of countries and years
all_countries = gdp_structure_pivot['Country'].unique()  # Get all unique countries
all_years = list(range(2000, 2024))  # Define the desired range of years

# Create a full dataframe with all combinations of countries and years
full_index = pd.MultiIndex.from_product([all_countries, all_years], names=['Country', 'Year'])
gdp_structure_full = pd.DataFrame(index=full_index).reset_index()

# Merge the full dataframe with the existing data
gdp_structure_main = pd.merge(gdp_structure_full,gdp_structure_pivot, on=['Country', 'Year'], how='left')


print(gdp_structure_main)


          Country  Year Agriculture (% of GDP) Industry (% of GDP)  \
0     Afghanistan  2000                    ...                 ...   
1     Afghanistan  2001                    ...                 ...   
2     Afghanistan  2002              43.721344           21.690862   
3     Afghanistan  2003              41.990574            21.36508   
4     Afghanistan  2004              34.218299           25.200659   
...           ...   ...                    ...                 ...   
1171     Viet Nam  2019              11.784529           36.803137   
1172     Viet Nam  2020              12.655404           36.743713   
1173     Viet Nam  2021               12.60309           37.391519   
1174     Viet Nam  2022              11.955529           38.175376   
1175     Viet Nam  2023              11.960486           37.120102   

     Services (% of GDP)  
0                    ...  
1                    ...  
2              34.587794  
3              36.644346  
4              40.581042

In [118]:
#Merge all dataframes
# List of DataFrames to merge
dataframes = [emission_main, employment_share_main, energy_intensity_main, gdp_main, rd_main, gdp_structure_main]

# Initialize the main DataFrame with the first DataFrame in the list
df_economic = dataframes[0]

# Loop through the remaining DataFrames and merge them
for df in dataframes[1:]:
    df_economic = pd.merge(df_economic, df, on=['Country', 'Year'], how='left')

df_economic.head()

,Country,Year,CO2 Emissions per Manufacturing Value-Added (kg per constant 2015 USD),"CO2 Emissions per GDP (PPP, kg per constant 2017 USD)",CO2 Emissions (t '000),Total GHG Emissions (t '000 CO₂ equivalent),Employment in Agriculture (% of total employment),Employment in Industry (% of total employment),Employment in Services (% of total employment),Energy Intensity Measured in Terms of Primary Energy and GDP (MJ/$ 2017 PPP GDP),GDP at current prices (domestic currency),R&D Expenditure (% of GDP),Agriculture (% of GDP),Industry (% of GDP),Services (% of GDP)
0,Afghanistan,2000,NaN,NaN,1078120,14205325.27,...,...,...,1.5,...,...,...,...,...
1,Afghanistan,2001,NaN,NaN,1088638,12604759.75,69.574145,6.187289,24.238567,1.5,...,...,...,...,...
2,Afghanistan,2002,NaN,NaN,1403030,15255393.53,69.574208,6.187477,24.238315,1.4,202530591600,...,43.721344,21.690862,34.587794
3,Afghanistan,2003,NaN,NaN,1653207,15961032.7,69.574667,6.18702,24.238313,1.39,240670915300,...,41.990574,21.36508,36.644346
4,Afghanistan,2004,NaN,NaN,1292307,15572012.38,69.574784,6.187135,24.238081,1.21,267310072200.0,...,34.218299,25.200659,40.581042
